<a href="https://colab.research.google.com/github/Hanbin-git/kaggle/blob/main/%EC%A0%84%EC%B2%98%EB%A6%AC_%EB%B0%8F_%ED%95%99%EC%8A%B502.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import shutil
import zipfile

# 1. Drive에서 로컬로 복사 (빠름)
shutil.copy("/content/drive/MyDrive/open.zip", "/content/open.zip")

# 2. 압축 풀기
with zipfile.ZipFile("/content/open.zip", "r") as zip_ref:
    zip_ref.extractall("/content/")

# 3. 확인
!ls /content/train


컨티넨탈_10세대_2017_2019	     A_클래스_W177_2020_2025
어코드_10세대_2018_2022		     B_클래스_W246_2013_2018
1시리즈_F20_2013_2015		     뉴_스타일_코란도_C_2017_2019
1시리즈_F20_2016_2019		     프리우스_C_2018_2020
1시리즈_F40_2020_2024		     뉴_CC_2012_2016
그랜드카니발_2006_2010		     CLA_클래스_C117_2014_2019
2008_2015_2017			     CLA_클래스_C118_2020_2025
에쿠스_신형_2010_2015		     CLE_클래스_C236_2024_2025
파나메라_2010_2016		     CLS_클래스_C257_2019_2023
뉴_제타_2011_2016		     CLS_클래스_W218_2012_2017
뉴_카이엔_2011_2018		     아반떼_하이브리드_CN7_2021_2023
엑센트_신형_2011_2019		     아반떼_CN7_2021_2023
스파크_2012_2015		     더_뉴_아반떼_CN7_2023_2025
쿠퍼_컨트리맨_2012_2015		     CT6_2016_2018
올_뉴_모닝_2012_2015		     C_클래스_W204_2008_2015
아베오_2012_2016		     C_클래스_W205_2015_2021
말리부_2012_2016		     C_클래스_W206_2022_2024
뉴_티구안_2012_2016		     제네시스_DH_2014_2016
레이_2012_2017			     쏘나타_DN8_2020_2023
올란도_2012_2018		     쏘나타_디_엣지_DN8_2024_2025
더_뉴_파사트_2012_2019		     e_트론_2020_2023
트랙스_2013_2016		     E_PACE_2018_2020
콰트로포르테_2014_2016		     EQ900_2016_2018
더_뉴_아반떼_2014_2016		     E

In [2]:
import os
import glob
import torch
import timm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import StratifiedKFold


In [3]:
# train 경로
train_dir = '/content/train'

# test 경로
test_dir = '/content/test'

# sample_submission (추론 때 사용)
sample_submission_path = '/content/sample_submission.csv'


In [4]:
import torch
import gc

def show_memory_status():
    allocated = torch.cuda.memory_allocated() / (1024 ** 2)  # MB 단위
    reserved = torch.cuda.memory_reserved() / (1024 ** 2)    # MB 단위
    print(f"📊 현재 GPU 메모리 상태: Allocated = {allocated:.2f} MB | Reserved = {reserved:.2f} MB")

# 현재 CUDA 사용 가능 여부 확인
if torch.cuda.is_available():
    print("🔍 초기화 전 GPU 메모리 상태:")
    show_memory_status()

    # GPU 캐시 및 메모리 초기화
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()

    print("\n🧹 GPU 메모리 초기화 완료")
    print("🔍 초기화 후 GPU 메모리 상태:")
    show_memory_status()
else:
    print("❌ CUDA 사용 불가")


🔍 초기화 전 GPU 메모리 상태:
📊 현재 GPU 메모리 상태: Allocated = 0.00 MB | Reserved = 0.00 MB

🧹 GPU 메모리 초기화 완료
🔍 초기화 후 GPU 메모리 상태:
📊 현재 GPU 메모리 상태: Allocated = 0.00 MB | Reserved = 0.00 MB


In [5]:
# 전체 crop
from PIL import Image
import os
import glob

# 원본 및 crop 저장 경로
input_root = "/content/train"
output_root = "/content/train_crop"

# 부위별 crop 함수 정의
def crop_views(image):
    w, h = image.size
    return {
        "original": image,
        "logo": image.crop((w*0.3, h*0.0, w*0.7, h*0.25)),      # 상단 로고
        "wheel": image.crop((w*0.1, h*0.6, w*0.9, h*0.95)),     # 하단 휠
        "interior": image.crop((w*0.25, h*0.3, w*0.85, h*0.8))  # 대략적인 실내
    }

# 전처리 루프
for class_dir in os.listdir(input_root):
    img_paths = glob.glob(os.path.join(input_root, class_dir, "*.jpg"))
    for img_path in img_paths:
        try:
            image = Image.open(img_path).convert("RGB")
            cropped = crop_views(image)
            for view_name, view_img in cropped.items():
                save_dir = os.path.join(output_root, view_name, class_dir)
                os.makedirs(save_dir, exist_ok=True)
                fname = os.path.basename(img_path)
                view_img.save(os.path.join(save_dir, fname))
        except Exception as e:
            print(f"⚠️ 에러 발생: {img_path} - {e}")


In [6]:
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch

class MultiViewDatasetWithColor(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.views = ["original", "logo", "wheel", "interior"]
        self.transform = transform

        self.image_info = []
        for label_idx, class_name in enumerate(sorted(os.listdir(os.path.join(root_dir, "original")))):
            for fname in os.listdir(os.path.join(root_dir, "original", class_name)):
                sample = {
                    "views": {view: os.path.join(root_dir, view, class_name, fname) for view in self.views},
                    "label": label_idx
                }
                self.image_info.append(sample)

    def __len__(self):
        return len(self.image_info)

    def __getitem__(self, idx):
        sample = self.image_info[idx]
        images = {view: Image.open(path).convert("RGB") for view, path in sample["views"].items()}

        color_feats = []
        if self.transform:
            for view, img in images.items():
                transformed_img = self.transform(img)
                images[view] = transformed_img
                mean_rgb = transformed_img.view(3, -1).mean(dim=1)  # [3]
                color_feats.append(mean_rgb)

        color_feat = torch.cat(color_feats, dim=0)  # [12]

        return images, color_feat, sample["label"]

# ✅ Transform 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ✅ Dataset 및 DataLoader 초기화
dataset = MultiViewDatasetWithColor(root_dir="/content/train_crop", transform=transform)
loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2)


In [7]:
import torch
import torch.nn as nn
import timm

class MultiInputEffNetB5_WithColor(nn.Module):
    def __init__(self, num_classes=396, pretrained=True):
        super(MultiInputEffNetB5_WithColor, self).__init__()
        self.views = ["original", "logo", "wheel", "interior"]

        # 각 view마다 EfficientNet-B5 encoder (num_classes=0: feature extractor로 사용)
        self.encoders = nn.ModuleDict({
            view: timm.create_model("efficientnet_b5", pretrained=pretrained, num_classes=0)
            for view in self.views
        })

        # 최종 classifier
        self.classifier = nn.Sequential(
            nn.Linear(len(self.views) * (2048 + 3), 1024),  # 2051 x 4 = 8204
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, num_classes)
        )

    def forward(self, images, rgb_means):  # rgb_means: [B, 12]
        B = rgb_means.size(0)
        feats = []

        for i, view in enumerate(self.views):
            img_feat = self.encoders[view](images[view])             # [B, 2048]
            rgb_feat = rgb_means[:, i*3:(i+1)*3].to(img_feat.device) # [B, 3]
            concat_feat = torch.cat([img_feat, rgb_feat], dim=1)     # [B, 2051]
            feats.append(concat_feat)

        all_feat = torch.cat(feats, dim=1)  # [B, 8204]
        out = self.classifier(all_feat)
        return out


In [8]:
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import os
import torch

class CustomMultiViewDataset(Dataset):
    def __init__(self, root_dir, view_names, transform=None):
        self.root_dir = root_dir  # e.g., '/content/train_crop'
        self.view_names = view_names  # e.g., ['original', 'logo', 'wheel', 'interior']
        self.transform = transform

        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(os.path.join(root_dir, view_names[0])))
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}

        for cls in self.classes:
            cls_dir = os.path.join(root_dir, view_names[0], cls)
            for img_name in os.listdir(cls_dir):
                if img_name.endswith(".jpg"):
                    self.image_paths.append(img_name)
                    self.labels.append(self.class_to_idx[cls])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = self.image_paths[idx]
        label = self.labels[idx]
        views = {}
        rgb_features = []

        for view in self.view_names:
            img_path = os.path.join(self.root_dir, view, self.classes[label], img_name)
            img = Image.open(img_path).convert("RGB")

            # RGB 평균값 계산 (transform 적용 전)
            np_img = np.array(img) / 255.0  # [H, W, 3], normalize to 0-1
            mean_rgb = np_img.mean(axis=(0, 1))  # shape: (3,)
            rgb_features.extend(mean_rgb.tolist())

            # transform 적용
            if self.transform:
                img = self.transform(img)

            views[view] = img

        # RGB 평균값을 torch.Tensor로 변환 [12]
        rgb_tensor = torch.tensor(rgb_features, dtype=torch.float32)

        return views, rgb_tensor, label


In [33]:
from torchvision import transforms
from torch.utils.data import DataLoader

# 🔧 EfficientNet-B5에 맞춘 전처리
train_transform = transforms.Compose([
    transforms.Resize((456, 456)),  # EfficientNet-B5 입력 사이즈
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ✅ RGB 평균 포함 Custom Dataset 생성
train_dataset = CustomMultiViewDataset(
    root_dir='/content/train_crop',
    view_names=['original', 'logo', 'wheel', 'interior'],
    transform=train_transform
)

# ✅ DataLoader 설정
train_loader = DataLoader(
    train_dataset,
    batch_size=2,         # 메모리 여유 시 8 이상으로 가능
    shuffle=True,
    num_workers=2,
    pin_memory=True       # 성능 향상 (가능할 경우)
)

# ✅ 배치 unpacking 예시
# for views, rgb_tensor, labels in train_loader:
#     print(views['original'].shape)  # torch.Size([4, 3, 456, 456])
#     print(rgb_tensor.shape)         # torch.Size([4, 12])
#     print(labels.shape)             # torch.Size([4])


In [34]:
import torch
import torch.nn as nn
import timm

class MultiInputEffNetB5_WithColor(nn.Module):
    def __init__(self, num_classes=396, pretrained=True):
        super(MultiInputEffNetB5_WithColor, self).__init__()
        self.views = ["original", "logo", "wheel", "interior"]

        self.encoders = nn.ModuleDict({
            view: timm.create_model("efficientnet_b5", pretrained=pretrained, num_classes=0)
            for view in self.views
        })

        self.classifier = nn.Sequential(
            nn.Linear(len(self.views) * 2051, 1024),  # 2048 feature + 3 RGB 평균
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, num_classes)
        )

    def forward(self, images, rgb_means):
        feats = []
        for view in self.views:
            img_feat = self.encoders[view](images[view])              # [B, 2048]
            start_idx = self.views.index(view) * 3
            end_idx = start_idx + 3
            rgb_feat = rgb_means[:, start_idx:end_idx].to(img_feat.device)  # [B, 3]
            feat_concat = torch.cat([img_feat, rgb_feat], dim=1)      # [B, 2051]
            feats.append(feat_concat)
        all_feats = torch.cat(feats, dim=1)                           # [B, 8204]
        return self.classifier(all_feats)



In [32]:
import gc

def show_memory_status():
    allocated = torch.cuda.memory_allocated() / (1024 ** 2)
    reserved = torch.cuda.memory_reserved() / (1024 ** 2)
    print(f"📊 GPU 메모리: Allocated = {allocated:.2f}MB | Reserved = {reserved:.2f}MB")

if torch.cuda.is_available():
    print("🔍 초기화 전 메모리 상태:")
    show_memory_status()

    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()

    print("✅ GPU 메모리 초기화 완료")
    print("🔍 초기화 후 메모리 상태:")
    show_memory_status()


🔍 초기화 전 메모리 상태:
📊 GPU 메모리: Allocated = 39678.02MB | Reserved = 39994.00MB
✅ GPU 메모리 초기화 완료
🔍 초기화 후 메모리 상태:
📊 GPU 메모리: Allocated = 19309.58MB | Reserved = 39994.00MB


In [35]:
# ✅ CUDA가 가능한 경우 GPU 사용
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 이후에 model 선언
model = MultiInputEffNetB5_WithColor(num_classes=396, pretrained=True).to(device)


In [36]:
import torch.nn as nn
import torch.optim as optim

# 손실 함수
criterion = nn.CrossEntropyLoss()

# 옵티마이저
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)

# (선택) learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

# ✅ 수정된 학습 함수
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for views, rgb_tensor, labels in dataloader:
        # 이미지와 RGB 평균 모두 device로 이동
        for key in views:
            views[key] = views[key].to(device)
        rgb_tensor = rgb_tensor.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # 모델 forward
        outputs = model(views, rgb_tensor)

        # 손실 계산 및 역전파
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 통계 기록
        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total
    return avg_loss, accuracy


In [37]:
EPOCHS = 15  # 최대 epoch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Early Stopping 설정
best_loss = float('inf')
patience = 5  # 성능이 향상되지 않아도 참는 횟수
counter = 0

for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    scheduler.step()

    print(f"[Epoch {epoch+1}] ✅ Loss: {train_loss:.4f} | Accuracy: {train_acc:.4f}")

    # Early stopping 조건 확인
    if train_loss < best_loss - 1e-4:  # 최소 개선량
        best_loss = train_loss
        counter = 0
        torch.save(model.state_dict(), "best_model.pth")  # 모델 저장
        print("📌 Model improved. Saved!")
    else:
        counter += 1
        print(f"⏳ No improvement. EarlyStopping patience: {counter}/{patience}")
        if counter >= patience:
            print("🛑 Early stopping triggered.")
            break


KeyboardInterrupt: 